**Data Mining Project, midterm 2021/2022**




**Authors:** Niko Dalla Noce, Alessandro Ristori, Giuseppe Lombardi

#**Task 1: Data Understanding and Data Preparation**

##**Task 1.1 Data Understanding**

**Importing libraries and extracting datasets**

In [27]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy

import zipfile
with zipfile.ZipFile("prj_data.zip", 'r') as zip_ref:
     zip_ref.extractall()

**Load the datasets**

Read the data from the three csv files, each one is assigned to a different dataframe for now.

In [28]:
df_male = pd.read_csv("male_players.csv", sep=",")  # dataframe of male players
df_female = pd.read_csv("female_players.csv", sep=",")  # dataframe of female players
df_matches = pd.read_csv("tennis_matches.csv", sep=",")  # matches dataframe

I dataframe dei giocatori contengono solamente nome e cognome dei giocatori (la grande maggioranza sono ormai ritirati dalle competizioni e non sono presenti in alcun record del dataframe dei match. Per questo abbiamo deciso di effetturare principlamente il nostro lavoro sul dataframe dei match.

In [29]:
df_matches.head()

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
0,0,2019-M020,Brisbane,Hard,32.0,A,20181231.0,300.0,105453.0,NaN,Kei Nishikori,R,178.0,JPN,95.000000,106421.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3.0,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,3928.0,742618.69
1,1,2019-M020,Brisbane,Hard,32.0,A,20181231.0,299.0,106421.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,104542.0,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,7-6(6) 6-2,3.0,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,3928.0,742618.69
2,2,2019-M020,Brisbane,Hard,32.0,A,20181231.0,298.0,105453.0,NaN,Kei Nishikori,R,178.0,JPN,29.004791,104871.0,NaN,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3.0,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,3928.0,742618.69
3,3,2019-M020,Brisbane,Hard,32.0,A,20181231.0,297.0,104542.0,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,200282.0,NaN,Alex De Minaur,R,183.0,AUS,19.868583,6-4 7-6(2),3.0,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,3928.0,742618.69
4,4,2019-M020,Brisbane,Hard,32.0,A,20181231.0,296.0,106421.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,105683.0,NaN,Milos Raonic,R,196.0,CAN,28.010951,6-7(2) 6-3 6-4,3.0,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,3928.0,742618.69


We found that the last record of the matches dataset (mainly the Taipei tournament) are missing a lot of values.

In [30]:
df_matches.tail()

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
186123,186123,NaN,NaN,Hard,32.0,NaN,20171113.0,250.0,NaN,NaN,NaN,NaN,NaN,NaN,26.206708,206075.0,NaN,NaN,NaN,NaN,NaN,26.90486,3-6 6-2 6-3,NaN,NaN,NaN,NaN,1.0,NaN,49.0,38.0,16.0,NaN,7.0,NaN,1.0,3.0,93.0,NaN,NaN,NaN,NaN,7.0,11.0,NaN,322.0,280.0,183.0,NaN,NaN
186124,186124,NaN,Taipei,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Veronika Kudermetova,NaN,175.0,NaN,NaN,214562.0,WC,Ingrid Neel,NaN,NaN,USA,NaN,7-6(3) 6-2,NaN,NaN,71.0,10.0,8.0,63.0,NaN,32.0,NaN,10.0,2.0,5.0,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,255.0,217.0,704.0,NaN,NaN,NaN
186125,186125,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,Dalila Jakupovic,NaN,NaN,NaN,26.642026,201674.0,NaN,NaN,R,NaN,UZB,NaN,NaN,3.0,NaN,70.0,NaN,2.0,55.0,NaN,NaN,NaN,NaN,6.0,8.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
186126,186126,NaN,NaN,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Vitalia Diatchenko,NaN,170.0,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-6 6-2 6-4,3.0,NaN,115.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,0.0,75.0,NaN,NaN,13.0,NaN,3.0,NaN,176.0,NaN,NaN,NaN,3535.0,NaN
186127,186127,NaN,Taipei,Hard,32.0,C,20171113.0,NaN,NaN,NaN,Priscilla Hon,NaN,NaN,AUS,19.512663,NaN,NaN,NaN,NaN,NaN,USA,NaN,6-4 6-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,NaN,NaN,43.0,25.0,9.0,10.0,4.0,8.0,NaN,NaN,181.0,322.0,3535.0,638954.72


In [31]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186128 entries, 0 to 186127
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          186128 non-null  int64  
 1   tourney_id          186073 non-null  object 
 2   tourney_name        186103 non-null  object 
 3   surface             185940 non-null  object 
 4   draw_size           186099 non-null  float64
 5   tourney_level       186099 non-null  object 
 6   tourney_date        186100 non-null  float64
 7   match_num           186101 non-null  float64
 8   winner_id           186073 non-null  float64
 9   winner_entry        25827 non-null   object 
 10  winner_name         186101 non-null  object 
 11  winner_hand         186082 non-null  object 
 12  winner_ht           49341 non-null   float64
 13  winner_ioc          186099 non-null  object 
 14  winner_age          183275 non-null  float64
 15  loser_id            186100 non-nul

In [32]:
df_matches.dtypes

Unnamed: 0              int64
tourney_id             object
tourney_name           object
surface                object
draw_size             float64
tourney_level          object
tourney_date          float64
match_num             float64
winner_id             float64
winner_entry           object
winner_name            object
winner_hand            object
winner_ht             float64
winner_ioc             object
winner_age            float64
loser_id              float64
loser_entry            object
loser_name             object
loser_hand             object
loser_ht              float64
loser_ioc              object
loser_age             float64
score                  object
best_of               float64
round                  object
minutes               float64
w_ace                 float64
w_df                  float64
w_svpt                float64
w_1stIn               float64
w_1stWon              float64
w_2ndWon              float64
w_SvGms               float64
w_bpSaved 

**First issue:** la data del torneo è nel formato sbagliato come anche lo sono tutti gli attributi riguardanti le statistiche dei giocatori e le loro informazioni.

In [33]:
df_matches['tourney_date']= df_matches['tourney_date'].astype(pd.Int32Dtype(), errors='ignore').astype(pd.StringDtype())
df_matches['tourney_date'] = pd.to_datetime(df_matches['tourney_date'], errors='coerce', format='%Y%m%d')
df_matches.iloc[:, [4, 7, 8, 12, 15, 19, 23]] = df_matches.iloc[:, [4, 7, 8, 12, 15, 19, 23]].astype("Int64", errors="ignore")
df_matches.iloc[:, 25:49] = df_matches.iloc[:, 25:49].astype("Int64", errors="ignore")
df_matches.dtypes

Unnamed: 0                     int64
tourney_id                    object
tourney_name                  object
surface                       object
draw_size                      Int64
tourney_level                 object
tourney_date          datetime64[ns]
match_num                      Int64
winner_id                      Int64
winner_entry                  object
winner_name                   object
winner_hand                   object
winner_ht                      Int64
winner_ioc                    object
winner_age                   float64
loser_id                       Int64
loser_entry                   object
loser_name                    object
loser_hand                    object
loser_ht                       Int64
loser_ioc                     object
loser_age                    float64
score                         object
best_of                        Int64
round                         object
minutes                        Int64
w_ace                          Int64
w

**Another issue:** dobbiamo eliminare quei record di cui non possiamo sapere né chi sia il vincitore né chi sia il perdente.

In [34]:
len_df_matches_previous = len(df_matches)
df_matches = df_matches.dropna(subset=["winner_name", "loser_name", "winner_id", "loser_id"], how="all")
print("{0} records from the matches dataframe were dropped since we coulnd't find any winner or loser".format(len_df_matches_previous-len(df_matches)))

5 records from the matches dataframe were dropped since we coulnd't find any winner or loser


In [52]:
df_matches.corr()

,Unnamed: 0,draw_size,match_num,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,best_of,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
Unnamed: 0,1.000000,-0.295322,-0.107764,0.284440,-0.123437,-0.156183,0.286927,-0.088326,-0.135181,-0.112521,-0.047323,-0.081921,-0.013361,-0.060045,-0.062510,-0.079890,-0.038855,-0.029392,-0.009178,-0.008738,-0.074958,-0.000226,-0.062421,-0.062922,-0.072158,-0.046882,-0.025756,-0.013241,-0.008301,0.311681,-0.227905,0.266397,-0.228344,-0.284925,-0.254112
draw_size,-0.295322,1.000000,0.270215,-0.211249,0.018760,0.199777,-0.218734,-0.007236,0.206215,0.378014,0.124134,0.072907,0.044526,0.141071,0.140390,0.147832,0.100516,0.088880,0.060899,0.072463,0.065323,0.034328,0.145275,0.141520,0.127084,0.095608,0.079876,0.067465,0.093134,-0.317525,0.329775,-0.316875,0.284922,0.929342,0.842169
match_num,-0.107764,0.270215,1.000000,-0.285763,0.069320,0.162903,-0.289177,0.049188,0.177078,0.032166,0.002851,-0.024621,-0.020520,0.004007,0.010981,-0.000645,-0.009208,0.002517,0.010247,0.015144,-0.014820,-0.025771,0.000822,0.005911,-0.001256,-0.003144,-0.000268,0.002265,0.004627,-0.206572,0.127164,-0.193370,0.129871,0.271362,0.183235
winner_id,0.284440,-0.211249,-0.285763,1.000000,-0.503485,-0.495656,0.896243,-0.541209,-0.347701,-0.195736,-0.026169,-0.255186,0.114304,-0.039336,-0.022155,-0.132997,-0.146201,-0.131417,0.093650,0.164339,-0.243094,0.100396,-0.063306,-0.033538,-0.142336,-0.158258,-0.126530,0.050753,0.168187,0.383267,-0.174511,0.291999,-0.131044,-0.197485,-0.180254
winner_ht,-0.123437,0.018760,0.069320,-0.503485,1.000000,0.118012,-0.510127,0.412755,0.131299,0.133758,0.038335,0.472399,-0.003408,0.065516,0.046401,0.180422,0.141881,0.154263,-0.105326,-0.176092,0.207010,-0.061815,0.087748,0.075303,0.185233,0.168727,0.144351,-0.055027,-0.181762,-0.046908,0.032934,-0.026052,0.000771,0.018208,0.021137
winner_age,-0.156183,0.199777,0.162903,-0.495656,0.118012,1.000000,-0.355623,0.125827,0.235372,0.120443,0.044255,0.076619,-0.052949,0.031395,0.036824,0.063480,0.033300,0.049672,-0.013131,-0.025438,0.085523,-0.027435,0.044755,0.042857,0.060009,0.047297,0.049814,0.006789,-0.012787,-0.389688,0.249022,-0.287940,0.191483,0.187513,0.169054
loser_id,0.286927,-0.218734,-0.289177,0.896243,-0.510127,-0.355623,1.000000,-0.531784,-0.504599,-0.195170,-0.050814,-0.262999,0.101165,-0.067760,-0.049040,-0.152941,-0.163940,-0.150801,0.070912,0.137112,-0.252709,0.104066,-0.082596,-0.055690,-0.168307,-0.170572,-0.144286,0.046723,0.169612,0.374293,-0.169673,0.346377,-0.151154,-0.204755,-0.186972
loser_ht,-0.088326,-0.007236,0.049188,-0.541209,0.412755,0.125827,-0.531784,1.000000,0.114426,0.128670,0.058441,0.196962,-0.072453,0.084979,0.082801,0.182475,0.162714,0.166695,-0.074200,-0.143320,0.432064,0.021113,0.099706,0.067237,0.201385,0.161552,0.162524,-0.052793,-0.169571,-0.006053,0.001513,0.001487,-0.015248,-0.007844,-0.002867
loser_age,-0.135181,0.206215,0.177078,-0.347701,0.131299,0.235372,-0.504599,0.114426,1.000000,0.112163,0.056558,0.103815,-0.015292,0.051413,0.052219,0.072394,0.050179,0.063708,0.001819,-0.007434,0.078774,-0.058471,0.048178,0.055188,0.080361,0.045382,0.058025,0.002361,-0.029862,-0.341425,0.212624,-0.393911,0.243345,0.194749,0.177394
best_of,-0.112521,0.378014,0.032166,-0.195736,0.133758,0.120443,-0.195170,0.128670,0.112163,1.000000,0.265147,0.247027,0.077529,0.304079,0.289481,0.352471,0.279675,0.390461,0.081616,0.078920,0.210259,0.077584,0.334361,0.309059,0.312344,0.256065,0.376302,0.135855,0.162484,-0.140155,0.253459,-0.128484,0.156920,0.344592,0.325269


#**Questa parte è data preparation** Now let's create a new dataset for the players

In [35]:
players_w = df_matches[["winner_name"]].rename(columns={"winner_name":"Name"})  # player names extracted from the winners
players_l = df_matches[["loser_name"]].rename(columns={"loser_name":"Name"})  # player names extracted from the losers

# first we append the losers list to the winners one
df_players = players_w.append(players_l)

# the redundant records are dropped and a new dataframe is built from the list of players
df_players = df_players["Name"].unique()
df_players = pd.DataFrame(df_players, columns=["Name"])
print("{0}\n".format(df_players.info()))
print(df_players.isnull().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10105 entries, 0 to 10104
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    10104 non-null  object
dtypes: object(1)
memory usage: 79.1+ KB
None

Name    True
dtype: bool


There's only one record with no value, let's drop it.

In [36]:
df_players = df_players.dropna()
df_players.isnull().any()

Name    False
dtype: bool

The new dataframe has only one attribute, as the head() method shows.

In [37]:
df_players.head()

,Name
0,Kei Nishikori
1,Daniil Medvedev
2,Jo-Wilfried Tsonga
3,Jeremy Chardy
4,Alex De Minaur


Now, we need to know if a player is male or female, that's when df_male and df_female come to help us.

In [38]:
def players_from_sex(df_players_sex, df_players_matches):
  """
  Obtain the players of a specific sex given a dataset of that sex
  :param df_players_sex: players dataframe (M or F) from where we obtain the sex
  :param df_players_matches: players dataframe built from the matches
  :return players: list of players (M or F)
  :return index: players indexes (M or F)
  """
  players_name = list(df_players_sex["name"])
  players_surname = list(df_players_sex["surname"])
    
  #lista di players nome e cognome da df_players
  players = list()
  for n, s in zip(players_name, players_surname):
    players.append("{0} {1}".format(n, s))

  players = df_players_matches[df_players_matches["Name"].isin(players)]

  players_indexes = players.index

  return list(players["Name"]), players_indexes

In [39]:
[players_m, i_male_found] = players_from_sex(df_male, df_players)
[players_f, i_female_found] = players_from_sex(df_female, df_players)

# players for which we have found the sex
players_sex_found = players_m+players_f

# players that belong to both datasets
players_both_sex = list(set(players_m) & set(players_f))
print("Players found in both datasets (male and female):\n{0}\n".format(players_both_sex))

# players with no sex
players_sex_not_found = df_players[~df_players["Name"].isin(players_sex_found)]

# some stats about what we found
print("Number of male players: {0}".format(len(players_m)))
print("Number of female players: {0}".format(len(players_f)))
print("Number of players: {0}".format(len(df_players)))
print("Number of players with no sex known: {0}\n".format(len(players_sex_not_found)))
players_sex_not_found

Players found in both datasets (male and female):
['Kuan Yi Lee', 'Austin Smith', 'Yang Liu', 'Yi Liu', 'Ziyu Zhang', 'Di Zhao']

Number of male players: 3018
Number of female players: 7062
Number of players: 10104
Number of players with no sex known: 30



,Name
2,Jo-Wilfried Tsonga
31,Cristian Garin
53,Stan Wawrinka
86,Antoine Hoang
95,Taylor Fritz
104,Mackenzie McDonald
110,Diego Schwartzman
136,Frances Tiafoe
204,Pedro Martinez
230,Lloyd Harris


All of the players with no known sex are in one of the datasets (male or female) but their name is written differently (e.g.: uppercase in matches but undercase in males), so we need to assign them a sex by looking at the tournaments they took part.

In [40]:
# let's insert a new attribute (sex) and assign the right values
df_players.loc[i_male_found, 'Sex'] = 'M'
df_players.loc[i_female_found, 'Sex'] = 'F'
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10104 entries, 0 to 10104
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    10104 non-null  object
 1   Sex     10074 non-null  object
dtypes: object(2)
memory usage: 556.8+ KB


To obtain the sex of those players with no known sex we can take a look at the tournaments' levels they took part in, since there's only one type that is shared by both sexes.

In [41]:
# tournament levels for each sex
male_t = {'M', 'G', 'C', 'A', 'S', 'F'}
female_t = {'P', 'PM', 'I', 'T1', '15', '25', '60', '100', '80', '10', '50', '75'}

players_sex_not_found_i = players_sex_not_found.index
for player, i in zip(players_sex_not_found["Name"], players_sex_not_found_i):
  player_matches_info = df_matches[df_matches['loser_name']==player]
  player_t_levels = set(player_matches_info["tourney_level"].dropna())
  if len(player_t_levels.intersection(male_t)) > 0:
    sex = "M"
    players_m.insert(i, player)
  else:
    if len(player_t_levels.intersection(female_t)) > 0:
      sex = "F"
      players_f.insert(i, player)
    else:
      sex = "Not known"
  
  df_players.loc[i, "Sex"] = sex
  print("{0} {1}".format(player, sex))

Jo-Wilfried Tsonga M
Cristian Garin M
Stan Wawrinka M
Antoine Hoang M
Taylor Fritz M
Mackenzie McDonald M
Diego Schwartzman M
Frances Tiafoe M
Pedro Martinez M
Lloyd Harris M
Franko Skugor M
Juan Martin del Potro M
J.J. Wolf M
Daniel Elahi Galan M
Joao Menezes M
Jurabek Karimov M
Juan Pablo Varillas M
Holger Rune M
Khumoun Sultanov M
Evgenii Tiurnev M
Alona Fomina F
Zeynep  Sena Sarioglan F
Christopher O'Connell M
Sam Groth M
Daniel Munoz de la Nava M
Botic van de Zandschulp M
Alexandar Lazov M
David O'Hare M
Ben Patael M
Andres Artunedo Martinavarro M


Now there are no more missing values for the sex attribute and each player is either M or F.

In [42]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10104 entries, 0 to 10104
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    10104 non-null  object
 1   Sex     10104 non-null  object
dtypes: object(2)
memory usage: 556.8+ KB


We still need to work on those players who appear in both the male and female datasets.

In [43]:
players_both_sex = df_players[df_players["Name"].isin(players_both_sex)]
players_both_sex_i = players_both_sex.index
players_both_sex

,Name,Sex
1166,Di Zhao,F
2891,Austin Smith,F
3178,Kuan Yi Lee,F
4096,Yi Liu,F
4669,Ziyu Zhang,F
7811,Yang Liu,F


**Non funziona**

In [44]:
players_both_sex_i = players_both_sex.index
for player, i in zip(players_both_sex["Name"], players_both_sex_i):
  player_matches_info = df_matches[df_matches['loser_name']==player]
  player_t_levels = set(player_matches_info["tourney_level"].dropna())
  if len(player_t_levels.intersection(male_t)) > 0 & len(player_t_levels.intersection(female_t)) == 0:
    sex = "M"
    players_m.remove(player)
  else:
    if len(player_t_levels.intersection(female_t)) > 0 & len(player_t_levels.intersection(male_t)) == 0:
      sex = "F"
      players_f.remove(player)
    else:
      sex = "Not known"
  
  df_players.loc[i, "Sex"] = sex
  print("{0} {1}".format(player, sex))

Di Zhao F
Austin Smith M
Kuan Yi Lee M
Yi Liu F
Ziyu Zhang F
Yang Liu M


Every player has just one sex after our work.



In [45]:
print(len(players_f))
print(len(players_m))
print(len(df_players))

7061
3043
10104


#**Cambiare tutto da qui in poi**

Let's take a look to the players dataframes by calling the head() method, which shows the first five lines of the dataframes.

In [46]:
df_male.head()

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales


In [47]:
df_female.head()

,name,surname
0,Bobby,Riggs
1,X,X
2,Martina,Hingis
3,Mirjana,Lucic
4,Justine,Henin


Let's now use the info() method to obtain information on the two player datasets.

In [48]:
print("Male players dataframe info")
df_male.info()
print("\nFemale players dataframe info")
df_female.info()

Male players dataframe info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55208 entries, 0 to 55207
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     55031 non-null  object
 1   surname  55166 non-null  object
dtypes: object(2)
memory usage: 862.8+ KB

Female players dataframe info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46172 entries, 0 to 46171
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     44505 non-null  object
 1   surname  46172 non-null  object
dtypes: object(2)
memory usage: 721.6+ KB


As we can see from the previous infos, there are null values both in the male and female dataset.

In [49]:
print("Male players dataframe null values:\n{0}".format(df_male.isnull().any()))
print("\nFemale players dataframe null values:\n{0}".format(df_female.isnull().any()))

Male players dataframe null values:
name       True
surname    True
dtype: bool

Female players dataframe null values:
name        True
surname    False
dtype: bool


We need to drop such elements with null values.

In [50]:
df_male = df_male.dropna()
df_female = df_female.dropna()
print("Male players dataframe info:")
df_male.info()
print("\nFemale players dataframe info:")
df_female.info()

Male players dataframe info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 55031 entries, 0 to 55207
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     55031 non-null  object
 1   surname  55031 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB

Female players dataframe info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 44505 entries, 0 to 46171
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     44505 non-null  object
 1   surname  44505 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


There could be duplicates in the same dataset, but we know that there's a possibility that two players could share the same name and surname, so we won't change anything.

But, there are elements where the values are missing, or incomplete, but not null (look at df_female.head()), we need to remove such elements.

In [51]:
elements_not_to_keep = ["Unknown", "??"]
df_male = df_male[df_male["name"].str.len()>1]
df_male = df_male[df_male["surname"].str.len()>1]
df_male = df_male[~df_male["name"].isin(elements_not_to_keep)]
df_male = df_male[~df_male["surname"].isin(elements_not_to_keep)]
df_male.info()
# df_male[df_male["name"].isin(["??"])]
# df_male.where(df_male.name.str.len()==1)
# Ci sono altri elementi che non sono compatibili, andrebbero visti a mano

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53965 entries, 0 to 55207
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     53965 non-null  object
 1   surname  53965 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB
